In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_train = pd.read_csv('train.csv')
df_train.head()

,User,Track,Rating
0,1,0,27
1,4,0,10
2,8,0,30
3,10,0,30
4,11,0,14


In [58]:
#Import the train_test_split function
from sklearn.model_selection import train_test_split

#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = df_train.copy()
y = df_train['User']

#Split into training and test datasets, stratified along user_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)
print(X_train.shape)
print(X_test.shape)

(113249, 3)
(37750, 3)


In [9]:
data_matrix = df_train.pivot(index='User', columns='Track', values='Rating').fillna(9999)
data_matrix.shape

(44734, 184)

In [10]:
data_matrix.head(5)

Track,0,1,2,3,4,5,6,7,8,9,...,174,175,176,177,178,179,180,181,182,183
User,,,,,,,,,,,,,,,,,,,,,
0,9999.0,32.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
1,9999.0,9999.0,9999.0,49.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
3,9999.0,28.0,30.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
4,10.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
6,9999.0,54.0,9999.0,48.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0


In [11]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')

In [12]:
user_similarity.shape

(44734, 44734)

In [38]:
user_similarity[[0,1]]

array([[0.        , 0.00542031, 0.00271957, ..., 0.00539372, 0.00543065,
        0.0136014 ],
       [0.00542031, 0.        , 0.00815693, ..., 0.00538447, 0.0054214 ,
        0.01359207]])

In [17]:
user_similarity[0][1]

0.005420307243188982

In [61]:
X_train = X_train.reset_index(drop=True)
X_train.head()

,User,Track,Rating
0,28514,86,28
1,8164,17,68
2,50619,177,8
3,45023,169,29
4,6692,22,85


In [96]:
def getRecommendations(df_train, person):
    '''
    Gets recommendations for a person by using a weighted average
    of every other user's rankings
    '''

    totals = {}
    simSums = {}
    for other in df_train['User']:
        print(other)
    # Don't compare me to myself
        if other == person:
            continue
        row = df_train.loc[df_train['User']==person].index[0]
        col = df_train.loc[df_train['User']==other].index[0]
        print(row, col)
        sim = user_similarity[row][col]
        # Ignore scores of zero or lower
        if sim <= 0:
            continue
        for item in ['Track']:
            # Only score movies I haven't seen yet
            if (~(df_train.loc[df_train['User'] == person]['Track'].isin([item])).all(axis=None)) or (df_train.loc[df_train['User'] == person]['Track'] == 9999):
                # Similarity * Score
                totals.setdefault(item, 0)
                # The final score is calculated by multiplying each item by the
                #   similarity and adding these products together
                totals[item] += df_train.loc[other, item] * sim
                # Sum of similarities
                simSums.setdefault(item, 0)
                simSums[item] += sim
    # Create the normalized list
    rankings = [(total / simSums[item], item) for (item, total) in
                totals.items()]
    # Return the sorted list
    rankings.sort()
    rankings.reverse()
    return rankings

In [97]:
getRecommendations(X_train, 8164)

28514
1 0
8164
50619
1 2
45023
1 3
6692
1 4
23650
1 5
36122
1 6
49327
1 7
44229
1 8
10241
1 9
26004
1 10
24500
1 11
33902
1 12
40935
1 13
19492
1 14
18644
1 15
34192
1 16
17329
1 17
800
1 18
41222
1 19
41813
1 20
6207
1 21
48191
1 22
30091
1 23
21818
1 24
19604
1 25
36767
1 26
36028
1 27
44619
1 28
8385
1 29
18522
1 30
6766
1 31
21557
1 32
44038
1 33
2553
1 34
17195
1 35
48724
1 36
47227
1 37
29554
1 38
32725
1 39
19902
1 40
7631
1 41
43625
1 42
20111
1 43
25613
1 44
28155
1 45
20248
1 46
50746
1 47
33201
1 48
34938
1 49
34414
1 50
41080
1 51
10320
1 52
2643
1 53
21957
1 54
50449
1 55
45144
1 56
30307
1 57
18759
1 58
6684
1 59
33244
1 60
21487
1 61
35359
1 62
9024
1 63
14951
1 64
6449
1 65
44844
1 66
30030
1 67
28005
1 68
1846
1 69
38969
1 70
16315
1 71
17782
1 72
31808
1 73
30652
1 74
31975
1 75
3899
1 76
432
1 77
19335
1 78
38818
1 79
45105
1 80
34924
1 81
29857
1 82
25082
1 83
29913
1 84
33177
1 85
19984
1 86
3282
1 87
50381
1 88
48737
1 89
4100
1 90
23966
1 91
19909
1 92
33040
1 93

1 736
27424
1 737
153
1 738
42687
1 739
30161
1 740
3757
1 741
6541
1 742
50815
1 743
33504
1 744
18142
1 745
47274
1 746
32997
1 747
6251
1 748
19767
1 749
18884
1 750
18293
1 751
24521
1 752
16193
1 753
8760
1 754
26413
1 755
50574
1 756
29843
1 757
32574
1 758
9116
1 759
35495
1 760
9654
1 761
43322
1 762
6187
1 763
9404
1 764
41131
1 765
32723
1 766
50465
1 767
44874
1 768
28983
1 769
119
1 770
22432
1 771
34539
1 772
37929
1 773
30351
1 774
22515
1 775
17361
1 776
32550
1 777
41903
1 778
33192
1 779
17969
1 780
40959
1 781
7102
1 782
30280
1 783
6925
1 784
19380
1 785
16479
1 786
26453
1 787
25018
1 788
9673
1 789
49599
1 790
34158
1 791
50823
1 792
45201
1 793
34494
1 794
15139
1 795
10817
1 796
49270
1 797
35459
1 798
50649
1 799
50007
1 800
28129
1 228
46106
1 802
29870
1 803
43834
1 804
35288
1 805
30294
1 806
20088
1 807
40558
1 808
30907
1 809
5617
1 810
50523
1 811
33962
1 812
23375
1 813
24911
1 814
1872
1 815
24918
1 816
23303
1 817
17692
1 96
36043
1 819
44590
1 820
2303

1 1405
19860
1 1406
22163
1 1407
44649
1 1408
30379
1 1409
32091
1 1410
50665
1 955
46884
1 1412
22976
1 1413
5911
1 1414
18846
1 1415
23393
1 1416
35655
1 1417
21574
1 1418
30591
1 1419
43690
1 1420
31020
1 1421
29752
1 1422
50281
1 1423
32671
1 1424
9873
1 1425
30024
1 1426
8749
1 1427
20420
1 1428
34727
1 1429
49469
1 1430
17875
1 1431
35248
1 1432
2537
1 1433
10046
1 1434
4911
1 1435
42803
1 1436
41792
1 1437
41528
1 1438
36471
1 1439
29809
1 1440
16824
1 1441
9741
1 1442
22059
1 1443
50033
1 1444
23329
1 1445
30247
1 1446
11757
1 1447
27908
1 1448
10935
1 1449
7957
1 1450
25293
1 1451
26668
1 1452
9122
1 1453
32868
1 1454
24697
1 1455
22003
1 1456
2643
1 53
38467
1 1458
27145
1 1459
44736
1 1460
767
1 1461
12993
1 1462
24485
1 1463
14474
1 1464
21998
1 1465
9618
1 1466
18204
1 1467
8147
1 1468
5900
1 1469
37839
1 1470
28618
1 1471
39890
1 1472
31836
1 1473
27369
1 1474
5329
1 1475
36552
1 1122
6068
1 1477
5350
1 1478
3540
1 1479
18264
1 1480
9625
1 1481
16976
1 1482
21342
1 506
16

50031
1 2131
30535
1 2132
50051
1 2133
6493
1 2134
19916
1 2135
43251
1 2136
40125
1 2137
28714
1 2138
44294
1 2139
29618
1 2140
49603
1 2141
9269
1 2142
17864
1 2143
41958
1 1670
25532
1 2145
33128
1 2146
27998
1 2147
44463
1 2148
30789
1 2149
33059
1 2150
39606
1 2151
44794
1 2152
30504
1 2153
20799
1 2154
29020
1 2155
18862
1 2156
47966
1 2157
20797
1 2158
32847
1 2159
20258
1 2160
31958
1 2161
10791
1 2162
33255
1 2163
5656
1 2164
50251
1 2165
7381
1 2166
32718
1 2167
37562
1 2168
32259
1 634
5552
1 2170
3488
1 2171
6909
1 2172
34705
1 2173
20857
1 2174
4745
1 2175
31605
1 2176
17875
1 1431
24175
1 2178
27889
1 383
21137
1 2180
49788
1 2181
28125
1 824
16244
1 2183
14520
1 2184
26353
1 2185
33969
1 2186
26996
1 2187
6338
1 2188
34109
1 2189
2943
1 2190
13178
1 2191
44038
1 33
37435
1 2193
50605
1 2194
44580
1 2195
50750
1 2196
5212
1 2197
21650
1 2198
22592
1 2199
26318
1 2200
42081
1 2201
22014
1 2202
3475
1 2203
20498
1 2204
44481
1 2205
37544
1 2206
2576
1 2207
8982
1 2208
16944

1 2772
18364
1 2773
45450
1 2774
6837
1 2775
28332
1 2776
28227
1 2777
6280
1 2778
23578
1 2779
35590
1 2780
12740
1 2781
31312
1 2782
32785
1 2783
32607
1 2784
25976
1 2785
6205
1 2786
24058
1 1653
13654
1 2788
22999
1 2391
38218
1 1861
4338
1 2791
27999
1 2792
18156
1 2793
21485
1 2794
24585
1 2795
28369
1 2796
24950
1 2797
49454
1 2798
42130
1 2799
16718
1 2800
29754
1 2801
29213
1 2802
49346
1 2803
23534
1 2804
19408
1 2805
47337
1 2806
28315
1 2807
21970
1 2808
17545
1 2809
34234
1 2810
26904
1 560
18503
1 2812
29609
1 2813
29528
1 2814
36801
1 2815
21363
1 2816
16671
1 2817
33721
1 2818
38994
1 249
42072
1 2820
2976
1 2821
8217
1 2822
6469
1 2823
6590
1 2824
22891
1 2825
23210
1 2826
36656
1 2827
37820
1 2828
5684
1 2829
1249
1 2830
50333
1 2831
10779
1 2832
10498
1 2833
39725
1 2834
33054
1 2835
17169
1 2836
24069
1 2837
30149
1 2838
28963
1 2839
11186
1 2840
28050
1 2841
38972
1 2842
22124
1 2843
8534
1 2844
30631
1 2845
3848
1 2448
42790
1 2847
46066
1 2848
19842
1 2849
22142


...

1 20095
17624
1 42627
31130
1 40701
28254
1 31784
43082
1 3732
49762
1 42631
46125
1 42632
29565
1 29643
28122
1 8790
21548
1 42635
29364
1 14671
43103
1 42637
24148
1 4586
4363
1 42639
28171
1 42640
2481
1 42641
7900
1 42642
631
1 42643
3478
1 42644
26026
1 27101
33230
1 42646
47719
1 9938
44525
1 42648
33436
1 6700
50788
1 8332
31539
1 36194
2345
1 42652
13307
1 42653
24889
1 42654
44218
1 42655
22423
1 42656
39785
1 42657
3273
1 24156
18407
1 18298
45500
1 42660
12957
1 42661
29965
1 17204
6703
1 10334
28887
1 15694
3231
1 42665
18546
1 3741
44830
1 42667
33428
1 31340
50781
1 18574
29399
1 42670
23420
1 42671
40361
1 42672
22123
1 42673
40720
1 42674
26682
1 42675
14527
1 42676
16745
1 14595
7689
1 42678
13373
1 42679
19994
1 27196
25426
1 28293
31338
1 30111
22336
1 16942
47377
1 20925
30778
1 30120
7085
1 42686
20564
1 3797
18133
1 30789
923
1 42689
4049
1 42690
31958
1 2161
42003
1 15287
28768
1 39317
42415
1 29751
29966
1 1018
35348
1 12273
20281
1 42697
17759
1 13404
50338
1 4

1 43284
30825
1 43285
36634
1 33626
28393
1 9205
19916
1 2135
35760
1 5792
7756
1 520
19901
1 6725
1489
1 43292
5176
1 43293
19348
1 43294
3882
1 27348
22595
1 8082
30940
1 24353
29278
1 43298
9253
1 25421
29908
1 7083
40429
1 13994
29541
1 2465
13187
1 43303
19322
1 19911
44285
1 43305
10787
1 43306
34145
1 26721
43484
1 43308
50693
1 11713
26689
1 11961
29331
1 43311
42544
1 43312
35564
1 6299
38149
1 12825
24862
1 22264
23001
1 43316
13222
1 18526
6925
1 784
42265
1 43319
32287
1 9867
15868
1 43321
3590
1 9204
49172
1 34597
20481
1 43324
3488
1 2171
49799
1 19002
4218
1 13734
29902
1 20252
39215
1 43329
33342
1 43330
14237
1 43331
29674
1 19357
6155
1 43333
45229
1 12059
24879
1 43335
47681
1 43336
10381
1 43337
5651
1 26128
20126
1 25357
47242
1 43340
18037
1 22329
20087
1 7549
26235
1 43343
33330
1 43344
37134
1 43345
23714
1 11524
23467
1 43347
14056
1 43348
45096
1 36977
39165
1 43350
33063
1 43351
35066
1 43352
27566
1 14656
20270
1 43354
24115
1 43355
6503
1 43356
28318
1 4335

47989
1 8105
28617
1 43891
23642
1 822
42020
1 43893
7186
1 43894
7912
1 43895
1475
1 43896
34853
1 15187
23712
1 43898
33361
1 1764
32226
1 2738
50915
1 43901
16687
1 18453
36840
1 7782
43690
1 1420
42744
1 38102
20387
1 621
44237
1 43907
30574
1 43908
29288
1 43909
24204
1 33055
12730
1 43911
3229
1 43912
29568
1 33486
21374
1 23903
22099
1 3821
7744
1 31006
30781
1 43917
22357
1 1560
40459
1 2368
31711
1 35974
5336
1 3365
34948
1 16032
680
1 43923
17278
1 10957
15586
1 43925
11635
1 43926
34695
1 15332
30189
1 25130
39352
1 43929
17312
1 6940
32414
1 26050
41417
1 43932
34256
1 25583
26398
1 9788
31255
1 20107
41905
1 32413
28789
1 41415
42399
1 4500
15
1 38825
17368
1 30357
20324
1 24565
21721
1 43942
20469
1 1786
18308
1 17647
21771
1 43945
22984
1 13035
33007
1 6272
30900
1 37475
26284
1 43949
30519
1 38762
20991
1 12092
25938
1 33052
5392
1 20574
38058
1 43954
43840
1 37578
56
1 43956
24244
1 21910
24163
1 43958
16729
1 13651
40834
1 24410
10420
1 43961
2071
1 34165
23900
1 4396

368
1 44499
47504
1 15328
37537
1 44501
31438
1 44502
34810
1 44503
22532
1 696
24635
1 21503
4694
1 44506
2370
1 11496
15089
1 20161
19172
1 5506
29444
1 31852
11935
1 44511
7289
1 44512
31043
1 44513
26331
1 13083
4661
1 44515
18590
1 678
22512
1 44517
6458
1 44518
22723
1 34244
9288
1 44520
21304
1 11135
27231
1 44522
3901
1 16872
11597
1 31874
49270
1 797
24646
1 44526
45409
1 19158
29099
1 15141
8599
1 9823
26246
1 44530
973
1 44531
33812
1 1611
8331
1 44533
50723
1 23310
42705
1 44535
28732
1 29261
15681
1 44537
35254
1 44538
32921
1 5308
22949
1 44540
33330
1 43344
13006
1 44542
44599
1 44543
22977
1 2356
2902
1 44545
34911
1 11092
49736
1 12087
33098
1 5545
19711
1 6976
3461
1 15538
30789
1 2149
33125
1 44552
2925
1 32069
39953
1 35614
29452
1 9860
26021
1 44556
19313
1 7386
33504
1 744
43721
1 44559
10393
1 44560
44708
1 44561
14544
1 44562
24479
1 2899
11009
1 44564
23204
1 44497
17271
1 5033
29886
1 4398
22593
1 22352
31384
1 5650
21694
1 20828
6087
1 44571
20431
1 9679
3279

IndexError: index 44734 is out of bounds for axis 0 with size 44734

In [98]:
def recommend(track, min_rating = 38):
    i = int(df_train.index[df_train['Track'] == movie_title][0])
    target = df_p[i]
    similar_to_target = df_p.corrwith(target)
    corr_target = pd.DataFrame(similar_to_target, columns = ['PearsonR'])
    corr_target.dropna(inplace = True)
    corr_target = corr_target.sort_values('PearsonR', ascending = False)
    corr_target.index = corr_target.index.map(int)
    corr_target = corr_target.join(df_title).join(df_movie_summary)[['PearsonR', 'Track', 'Rating', 'mean']]
    print(corr_target[corr_target['count']>min_count][:10].to_string(index=False))